In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn import tree
from sklearn import datasets
from sklearn import linear_model
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from datetime import datetime
from sklearn.datasets import load_iris
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

**Read and clean data**

In [2]:
iris = load_iris()
df = pd.DataFrame(data=np.c_[iris["data"], iris["target"]],
                     columns=iris["feature_names"] + ["target"])

# Inputs: sepal length (cm),	sepal width (cm),	petal length (cm)
X = df[df.columns[0:3]]

# Output (dependent var) - in series format
y = df['petal width (cm)']

**Split into train/test** 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


**Define Decision Tree model type and fit**

In [4]:
tree_model = tree.DecisionTreeRegressor()

tree_model.fit(X_train, y_train)

DecisionTreeRegressor()

**DT Hyperparameter Tuning**

In [5]:
# This can take long 'cause runs down a lot of paths
grid_param_tree = {"max_depth" : range(1, 31),
                   "max_features" : range(1, len(df.columns))}

print(str(datetime.now()))
grid_search_tree = ms.GridSearchCV(tree_model,
                                   grid_param_tree,
                                   cv=4,
                                   scoring="r2").fit(X_train, y_train)
print(str(datetime.now()))

print("Best parameters:", grid_search_tree.best_params_,
      "\nBest Score:", grid_search_tree.best_score_)

2022-08-21 02:48:18.998299
2022-08-21 02:48:23.994392
Best parameters: {'max_depth': 3, 'max_features': 3} 
Best Score: 0.9234011452742836


**Define Random Forest model type and fit**

In [6]:
rf_model = ensemble.RandomForestRegressor(random_state=1, n_jobs=-1)
# print(rf_model)

rf_model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=1)

**RF Hyperparameter Tuning**

In [7]:
# This can take long 'cause runs down a lot of paths
grid_param_forest = {"max_depth" : range(1, 5),
                     "n_estimators" : range(10, 300, 20),
                     "max_features" : range(1, len(df.columns))}

print(str(datetime.now()))
grid_search_forest = ms.GridSearchCV(rf_model,
                                    grid_param_forest,
                                    cv=3,
                                    verbose=True,
                                    scoring="r2").fit(X_train, y_train)
print(str(datetime.now()))

print("Best parameters:", grid_search_forest.best_params_,
      "\nBest Score:", grid_search_forest.best_score_)

2022-08-21 02:48:24.349070
Fitting 3 folds for each of 240 candidates, totalling 720 fits
2022-08-21 02:54:33.127148
Best parameters: {'max_depth': 3, 'max_features': 3, 'n_estimators': 30} 
Best Score: 0.9424032781851945


**Predict each model**

In [8]:
prediction_data = [4, 3.2, 1.6]

tree_prediction = grid_search_tree.predict([prediction_data])
print("DT Petal width (cm) prediction for inputs:", prediction_data, "is", tree_prediction)

DT Petal width (cm) prediction for inputs: [4, 3.2, 1.6] is [0.17857143]


In [9]:
rf_prediction = grid_search_forest.predict([prediction_data])
print("RF Petal width (cm) prediction for inputs:", prediction_data, "is", rf_prediction)

RF Petal width (cm) prediction for inputs: [4, 3.2, 1.6] is [0.23835976]


**Average the models**

In [10]:
def model_ensemble(sl, sw, pl, tree_weight=0.5):
  tree_prediction = grid_search_tree.predict([[sl, sw, pl]])
  rf_prediction = grid_search_forest.predict([[sl, sw, pl]])
  return ((tree_weight * tree_prediction + (1 - tree_weight) * rf_prediction))[0]

In [11]:
print("Sepal Length = 4, Sepal width = 3.2, Petal length = 1.6")

# use DT weights = .2, .5, .8
for i in np.arange(.2, .81, .3):
  print("With Decision Tree weighting:", i * 100, "percent; ensemble Petal width (cm) prediction is:", model_ensemble(4, 3.2, 1.6, i))

Sepal Length = 4, Sepal width = 3.2, Petal length = 1.6
With Decision Tree weighting: 20.0 percent; ensemble Petal width (cm) prediction is: 0.22640209722154428
With Decision Tree weighting: 50.0 percent; ensemble Petal width (cm) prediction is: 0.20846559647775087
With Decision Tree weighting: 80.0 percent; ensemble Petal width (cm) prediction is: 0.1905290957339575


**Scoring**

In [12]:
tree_train_score = grid_search_tree.best_estimator_.score(X_train, y_train)
tree_train_error = 1 - tree_train_score

print("DT Score on data used in training =", tree_train_score, "==>error =", tree_train_error)

DT Score on data used in training = 0.9541826369835199 ==>error = 0.045817363016480095


In [13]:
tree_test_score = grid_search_tree.best_estimator_.score(X_test, y_test)
tree_test_error = 1 - tree_test_score

print("DT Score on test (unseen) data =", tree_test_score, "==>error =", tree_test_error)

DT Score on test (unseen) data = 0.8860711225314798 ==>error = 0.1139288774685202


In [14]:
grid_train_score = grid_search_forest.best_estimator_.score(X_train, y_train)
grid_train_error = 1 - grid_train_score

print("RF Best estimator score on data used in training =", grid_train_score, "==>error =", grid_train_error)

grid_test_score = grid_search_forest.best_estimator_.score(X_test, y_test)
grid_test_error = 1 - grid_test_score

print("RF Best estimator score on test (unseen) data =", grid_test_score, "==>error =", grid_test_error)

RF Best estimator score on data used in training = 0.9623596462965536 ==>error = 0.03764035370344643
RF Best estimator score on test (unseen) data = 0.9112091772954077 ==>error = 0.08879082270459226
